In [1]:
import sys

#Due to leftoevererrors in Nathan's python installation, some cleaning up occurs here
#sys.path.append("./code/")
#sys.path.remove('/usr/local/lib/python2.7/site-packages') 


import matplotlib.pyplot as plt
%matplotlib inline
import copy
import datetime
import pickle
import gzip
import os,glob
import time
import numpy as np
import pandas as pd
import operator
import random
from  scipy import stats



from scipy.stats import norm
from sklearn.neighbors import KernelDensity


import itertools
#sys.path


import regex as re
import datetime
import math
import time
import itertools

import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools
import plotly.figure_factory as ff

import plotly
plotly.tools.set_credentials_file(username='juliettapc', api_key='nM6iUdx6dGaOiPXQTwpP')   # go to: https://plot.ly/settings/api#/   for a new key if needed

########## to be able to plot offline (without sending the plots to the plotly server every time)
import plotly.offline as offline
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)
################



from IPython.core.display import display,HTML
display(HTML("<style>.container { width:100% !important; }</style>"))  # to make the notebook use the entire width of the browser


In [2]:
                                  

####  NEW FILE INCLUDING EARLY CITATIONS OF YOUNG REFERENCES:   ../data/df_reference_cite_plos_no_self-cit_one_ref_per_sect_ONLY_ARTICLES_early_cit.pkl
                                            
#%time df_merged = pickle.load(open('../data/df_reference_cite_plos_merged_simplified_added_more_columns_no_self-cit_one_ref_per_sect_ONLY_ARTICLES.pkl', 'rb'))



%time df_merged = pickle.load(open('../data/df_reference_cite_plos_no_self-cit_one_ref_per_sect_ONLY_ARTICLES_early_cit_and_after_accretion_time.pkl', 'rb'))
print ("done loading pickles", df_merged.shape)




df_merged = df_merged[df_merged['cite_count'] != -1]   # i dont know why, but there are 7 occurrences with value -1



plos_df = df_merged.drop_duplicates(subset=['paper_UT'])
print (plos_df.shape)



df_merged.head()

CPU times: user 4.68 s, sys: 1.27 s, total: 5.95 s
Wall time: 16.5 s
done loading pickles (5787634, 34)
(156558, 34)


,occurence,reference_UT,reference_rank,regex_sect_index,cite_count,ref_pub_year,paper_cite_count,plos_pub_year,sect_char_pos,sect_char_total,...,plos_article_type,num_cit_young_ref_by2009,num_cit_young_ref_by2010,num_cit_young_ref_by2011,num_cit_young_ref_by2012,num_cit_young_ref_by2013,num_cit_young_ref_by2009after8,num_cit_young_ref_by2008after8,num_cit_young_ref_by2010after8,num_cit_young_ref_by2007after8
0,1,A1995QY75100004,1,0,60.0,1995.0,2,2013.0,139,4029,...,@ Article,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,A1995QY75100004,1,3,60.0,1995.0,2,2013.0,494,5398,...,@ Article,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,000263911400006,3,0,5.0,2009.0,2,2013.0,142,4029,...,@ Article,0.0,0.0,NaN,NaN,NaN,5.0,5.0,5.0,5.0
5,1,000289279600018,4,0,29.0,2011.0,2,2013.0,269,4029,...,@ Article,0.0,0.0,0.0,3.0,NaN,39.0,39.0,39.0,39.0
6,3,000289279600018,4,3,29.0,2011.0,2,2013.0,3844,5398,...,@ Article,0.0,0.0,0.0,3.0,NaN,39.0,39.0,39.0,39.0


In [4]:
######  i create the new columns for fraction of references in each section (labelled 0 to 7)



%time plos_df = pickle.load(open('../data/plos_paper_dataframe_ONLY_ARTICLES_num_ref_sect_young_old.pkl', 'rb'))
print ("done loading plos_df", plos_df.shape)
plos_df_simplified = plos_df[['paper_UT','num_ref_section0','num_ref_section1','num_ref_section2','num_ref_section3','num_ref_section4','num_ref_section5','num_ref_section6','num_ref_section7',
    'fract_old_ref_section0', 'fract_old_ref_section1', 'fract_old_ref_section2', 'fract_old_ref_section3', 'fract_old_ref_section4', 'fract_old_ref_section5', 'fract_old_ref_section6', 'fract_old_ref_section7',\
    'fract_young_ref_section0', 'fract_young_ref_section1', 'fract_young_ref_section2', 'fract_young_ref_section3', 'fract_young_ref_section4', 'fract_young_ref_section5', 'fract_young_ref_section6', 'fract_young_ref_section7']]

# print ("plos_df_simplified", plos_df_simplified.shape)




#df_merged = pd.merge(df_merged, plos_df_simplified, on='paper_UT', how='left')
df_merged = pd.merge(df_merged, plos_df_simplified, on='paper_UT', how='left')
print ("df_merged", df_merged.shape)

# df_merged[['paper_UT','reference_UT','regex_sect_index','eff_num_ref','num_ref_section0','num_ref_section1','num_ref_section2','num_ref_section3','num_ref_section4','num_ref_section5','num_ref_section6','num_ref_section7']].head(100)




df_records_one_ref_per_plos = df_merged.drop_duplicates(subset=['paper_UT'])
print (df_records_one_ref_per_plos.shape)



# df_records_one_ref_per_plos.rename(columns={'paper_cite_count_x': 'paper_cite_count', 'plos_pub_year_x': 'plos_pub_year', 'ref_pub_year_x':'ref_pub_year','regex_sect_index_x':'regex_sect_index', 'total_refs_x':'total_refs'}, inplace=True)
# df_merged.rename(columns={'paper_cite_count_x': 'paper_cite_count', 'plos_pub_year_x': 'plos_pub_year', 'ref_pub_year_x':'ref_pub_year','regex_sect_index_x':'regex_sect_index', 'total_refs_x':'total_refs','reference_UT_x':'reference_UT','cite_count_x':'cite_count'}, inplace=True)


sorted(df_records_one_ref_per_plos.columns)

df_records_one_ref_per_plos.head()



###OJO!!!!! FALTA POR HACER BIEN EL MERGE PARA EVITAR REPETICION DE COLUMNAS!!!

CPU times: user 466 ms, sys: 114 ms, total: 580 ms
Wall time: 1.32 s
done loading plos_df (156610, 87)
df_merged (5787630, 58)
(156558, 58)


,occurence,reference_UT,reference_rank,regex_sect_index,cite_count,ref_pub_year,paper_cite_count,plos_pub_year,sect_char_pos,sect_char_total,...,fract_old_ref_section6,fract_old_ref_section7,fract_young_ref_section0,fract_young_ref_section1,fract_young_ref_section2,fract_young_ref_section3,fract_young_ref_section4,fract_young_ref_section5,fract_young_ref_section6,fract_young_ref_section7
0,1,A1995QY75100004,1,0,60.0,1995.0,2,2013.0,139,4029,...,0.0,0.0,0.000000,0.0,0.0,0.032258,0.0,0.0,0.0,0.0
31,1,000088374400026,1,0,477.0,2000.0,2,2015.0,543,2207,...,0.0,0.0,0.023256,0.0,0.0,0.023256,0.0,0.0,0.0,0.0
74,1,000313346100007,2,0,6.0,2013.0,1,2015.0,591,2474,...,0.0,0.0,0.181818,0.0,0.0,0.090909,0.0,0.0,0.0,0.0
85,1,000301376000017,2,0,35.0,2012.0,8,2012.0,762,2700,...,0.0,0.0,0.130435,0.0,0.0,0.086957,0.0,0.0,0.0,0.0
108,1,000232311300030,1,0,1687.0,2005.0,98,2008.0,77,3195,...,0.0,0.0,0.000000,0.0,0.0,0.040000,0.0,0.0,0.0,0.0


In [5]:
### FIGURE 2A, HEATMAP FOR NUMBER OF REFERENCES

       

lista_bin_names = ["Bottom", "Typical", "Good", "High", "Top"]
lista_sections = ["Introduction", "Methods", "Results", "Discussion"]




list_q=[0.3,0.6,.9,.99,1]    
    
#### i get the bins number of citation of the plos papers OJO!!!!! i want the same bins for all papers (so i calculate them before separating into sections but after all the preselections)

quantiles=sorted(list(df_records_one_ref_per_plos['paper_cite_count'].quantile(list_q).to_dict().items())) #mean 10.68 


dict_group_subset_data = {}
lista_bins_plos_citations=[]
old_value=0
for item in quantiles:
    try:
        pair=[old_value, int(item[1])]    
    except:  # if it is a nan:
        pair=[old_value, item[1]]
    
    lista_bins_plos_citations.append(pair)
    
    try:
        old_value = int(item[1])
    except:
        old_value = item[1]

print (lista_bins_plos_citations, df_records_one_ref_per_plos.shape)


lista_set_names = []

lista_text_z = []
list_of_lists =[]
cont_citation_groups=0
for pair_values in lista_bins_plos_citations:    # needs to correspond to    ["Bottom","Typical","Good","High","Top"]   
    #print ("-------",pair_values, cont_citation_groups, lista_bin_names[cont_citation_groups])
    
    minimo = pair_values[0]
    maximo = pair_values[1]   
    
    string_cit_group = lista_bin_names[cont_citation_groups]
    cont_citation_groups += 1
    

    df_select = df_records_one_ref_per_plos[(df_records_one_ref_per_plos['paper_cite_count'] >= minimo)  &  (df_records_one_ref_per_plos['paper_cite_count'] < maximo)]            
    
    list_q_cell = [.25,.5,.75]
    list_quantiles = []   # for [intro, methods, results, discussion]
    for column in ['num_ref_section0', 'num_ref_section1', 'num_ref_section2', 'num_ref_section3']:   #  intro  methods results dicussion
        quantiles_cell_aux  = sorted(list(df_select[column].quantile(list_q_cell).to_dict().items())) #mean 10.68 
        list_quantiles.append(quantiles_cell_aux)
        
                
        x1 = list(df_select[column])
        


        if column == 'num_ref_section0':                        
            group= "Introduction "+ string_cit_group       
            
        elif column ==  'num_ref_section1':                              
            group= "Methods "+ string_cit_group                
            
        elif column ==  'num_ref_section2':               
            group= "Results "+ string_cit_group          
                
        elif column ==  'num_ref_section3':               
            group= "Discussion "+ string_cit_group                   
            

        lista_set_names.append(group)   

        dict_group_subset_data[group]=x1
        
        
       
    print (pair_values, df_records_one_ref_per_plos.shape, df_select.shape, list_quantiles)
   


    #example   list_quantiles: [[(0.25, 7.0), (0.5, 12.0), (0.75, 17.0)],      [(0.25, 1.0), (0.5, 3.0), (0.75, 6.0)],      [(0.25, 0.0), (0.5, 0.0), (0.75, 2.0)],      [(0.25, 5.0), (0.5, 11.0), (0.75, 18.0)]]
    
    
     
    aux_list = [int(df_select.num_ref_section0.median()), int(df_select.num_ref_section1.median()), int(df_select.num_ref_section2.median()), int(df_select.num_ref_section3.median()) ]      # [intro, methods, results, discussion]
    
    
    
    lista_trios = []
    for lista_pares in list_quantiles:
        trio_valores = [ int(i[1]) for i in lista_pares]
        lista_trios.append(trio_valores)
    
    print (aux_list,"\n", lista_trios,'\n')
    list_of_lists.append(aux_list)
    
    
    
                                                                                                                                          
                                                                                                                                          
    aux_lista_text_z =  [str(lista_trios[i]).replace('[','').replace(']','').replace(', ', '-')   + ""     for i in range(len(aux_list))]     # [intro, methods, results, discussion]
  
    
    
   

    new_lista = [ item.split("-")[0] + "-<b>"+ item.split("-")[1] +"</b>-"+ item.split("-")[2] for item in aux_lista_text_z  ]  # i add bold font for the median value of the trio
    
    
    lista_text_z.append(new_lista)
    
  
    
    
    
print ("\n\n")
for lista in list_of_lists:
    print (lista)

print ("\n\n")
for lista in lista_text_z:
    print (lista)
 









fig_colorscale = [[0, '#ecbfe0'], [1, '#66475e']]   #   [[0, 'dcf0d2'], [1, '#205803']]   # if i give it a min and a max colors in HEX, it creates a gradient from one to another
fig_font_colors = [ '#3c3636', '#efecee']   #['#205803', '#dcf0d2']      # same for the annotation of the boxes (to make sure they are readable)
# fig_title_plot = "Median difference between publication year of plos and references in "+string_journal+string_plos_field+" papers from "+str(years)+string_isolated_ref+string_self_ref+string_code_categ+"<br>Number of occurrences: "+str(N_all)#+",   Number plos: "+str(N_plos)
# fig_filename = '../plots/annotated-heatmap_median_age_difference_plos_publ_year_vs_references_for_sections_and_subsect_by_citations_of_plos'





colorbar_string = 'Number of References'





#fig = ff.create_annotated_heatmap(z=lista_z50, x=lista_sections, y=lista_bin_names, annotation_text=lista_text_z, colorscale=fig_colorscale, font_colors=fig_font_colors,showscale=True, colorbar=dict(title=colorbar_string, titleside='right' ),)#, reversescale=True)

#colorscale='Viridis', 


    
fig = ff.create_annotated_heatmap(z=list_of_lists, x=lista_sections, y=lista_bin_names, annotation_text=lista_text_z, colorscale=fig_colorscale, font_colors=fig_font_colors, showscale=True, colorbar=dict(title=colorbar_string, titleside='right', ticks='outside' ))










#fig = ff.create_annotated_heatmap(z=lista_z50, x=lista_sections, y=lista_bin_names,  colorscale=fig_colorscale, font_colors=fig_font_colors,showscale=True)#, reversescale=True)
fig.layout.title = ""# fig_title_plot

fig['layout']['xaxis']['side'] = 'bottom'
fig.layout.xaxis.update({'title': 'Section'})


fig.layout.yaxis.update({'title': 'Impact Group'})




font_gral=25  # 20 if i wanna see it on the browser, 40 if i care about the png output
fig['layout']['font']['size'] = font_gral

      


font_gral=55  # 20 if i wanna see it on the browser, 40 if i care about the png output
fig['layout']['font']['size'] = font_gral


# Altering x axis7
#fig['layout']['xaxis']['tickfont']['family'] = 'Gill Sans MT'
fig['layout']['xaxis']['tickangle'] = 0
fig['layout']['yaxis']['tickangle'] = -90
fig['layout']['xaxis']['titlefont']['size'] = font_gral  #+10
fig['layout']['yaxis']['titlefont']['size'] = font_gral

fig['layout']['xaxis']['tickfont']['size'] = font_gral -7
fig['layout']['yaxis']['tickfont']['size'] = font_gral -15




fig['layout']['margin']=dict(
        l=200,
       # r=50,
        b=150,
        t=0,
        pad=15
    )







offline.plot(fig, auto_open=True, image = 'png', image_filename='testing_annotated_heatmap_color' ,image_width=2000, image_height=1200, filename='testing_annotated_heatmap_color.html', validate=True)


############################################
########################################################################
################################################


## COPIED CELL!!##############  I RUN THE TESTS FOR THE COMPARISON OF ALL PAIRS OF SUB-SETS:  (all pair-wise comparison cells in figure 2A)

#####  mann-whitney U   test



lista_indeces = [[1,1],[1,2],[1,3],[1,4],\
                 [2,1],[2,2],[2,3],[2,4],\
                 [3,1],[3,2],[3,3],[3,4],\
                 [4,1],[4,2],[4,3],[4,4],\
                 [5,1],[5,2],[5,3],[5,4]]

#fig_colorscale=  [ [0., '#f2f2f2'], [threshold_zero,'#cce6ff'], [0.9999, '#0059b3'], [1.,'#bdbdbd']]# '#ffffff']]  0: white,  .99999: blue,  1: grey
fig_colorscale=  [ [0., '#0059b3'], [.5,'#c7dcf1'], [1.,'#bdbdbd']] #  0 or anything significant: blue,   .5 or anithing NON signif: light-blue,     1: grey






lista_bin_names = ["Bottom","Typical","Good","High","Top"]
lista_sections  =  ['Intro', 'Methods', 'Results', 'Discussion']






tot_rows = 5
tot_cols = 4  
    
test = "MW"  # KS to test if the distributions are different or  MW for testing just whether the medians are different 
    

### i apply the bonferroni correction:  new p-_value_threshold  required for significance = old_p_value /Number of comparisons = 0.001 / (20*19/2)     
    


list_keys_macro = ['Introduction Top',    'Methods Top',    'Results Top',   'Discussion Top', \
                   'Introduction High',   'Methods High',  'Results High',  'Discussion High', \
                   'Introduction Good',   'Methods Good',   'Results Good',  'Discussion Good', \
                   'Introduction Typical', 'Methods Typical', 'Results Typical', 'Discussion Typical', \
                   'Introduction Bottom',  'Methods Bottom',  'Results Bottom', 'Discussion Bottom']
    
      
list_keys_heatmap = ['Introduction Bottom', 'Methods Bottom',  'Results Bottom', 'Discussion Bottom',\
                     'Introduction Typical', 'Methods Typical', 'Results Typical', 'Discussion Typical',\
                     'Introduction Good',   'Methods Good',   'Results Good',  'Discussion Good', \
                     'Introduction High',   'Methods High',  'Results High',  'Discussion High', \
                     'Introduction Top',   'Methods Top',    'Results Top',   'Discussion Top'  ]



threshold_zero = 0.0001 / (float(len(list_keys_macro))*float(len(list_keys_macro)-1)/2.)    # to round up to zero the very small p_values

 
#lista de sets en el orden obtenido en la celda anterior:

# ['Introduction Bottom',
#  'Methods Bottom',
#  'Results Bottom',
#  'Discussion Bottom',
#  'Introduction Typical',
#  'Methods Typical',
#  'Results Typical',
#  'Discussion Typical',
#  'Introduction Good',
#  'Methods Good',
#  'Results Good',
#  'Discussion Good',
#  'Introduction High',
#  'Methods High',
#  'Results High',
#  'Discussion High',
#  'Introduction Top',
#  'Methods Top',
#  'Results Top',
#  'Discussion Top']





lista_tot_datos=[]

total_cont =0
for i in list_keys_macro:
    lista_listas=[]
    aux_lista=[]
    cont=1
   
    for j in list_keys_heatmap:
             
        set1 = dict_group_subset_data[i]
        set2 = dict_group_subset_data[j]
        
        if test == "KS":
            p_value = stats.ks_2samp(set1, set2)[1] 
        elif test == "MW":
            p_value = stats.mannwhitneyu(set1, set2,  alternative='two-sided')[1]  
        
        
        if p_value <= threshold_zero:  #i round up to zero the very small p_values
                p_value =0.
                
        else:
            p_value = .5  ### I ONLY CARFE ABOUT WHEThER IT IS SIGNIFICANT OR NOT, I DONT CARE ABOUT THE EXACT P-VALUE ONCE IT IS NOT SIGNIFICANT
            
            
                
        if i == j:  # i single out manually the self comparison
            p_value=1.001  
            
            
            
        aux_lista.append(p_value)
        
        cont +=1
        
        if cont > tot_cols:
            lista_listas.append(aux_lista)                  
            aux_lista=[]
            cont=1
            
    
    
    total_cont +=1
    lista_tot_datos.append(lista_listas)
    
    
   


##############################

### I create an empty plot
fig_macro = None
fig_macro = tools.make_subplots(rows=tot_rows, cols=tot_cols, shared_xaxes=True, shared_yaxes=True, vertical_spacing = 0.01, horizontal_spacing = 0.01,   )


for i in range(len(lista_tot_datos)):
    datos = lista_tot_datos[i]
    
    cont_rows = lista_indeces[i][0]
    cont_cols = lista_indeces[i][1]
    
    trace1 = go.Heatmap(z=datos,
                       x=lista_sections,
                       y=lista_bin_names,                        
                       colorscale = fig_colorscale,
                       showscale=False,)
                        #reversescale=True, )#    )

   



    
    # i add each plot
    fig_macro.append_trace(trace1, cont_rows, cont_cols)


 



fontsize=32 
fig_macro['layout']['font'].update({'size': fontsize})






fig_macro['layout']['margin'].update({'l': 250})   # i add extra space to the margins of the plots
fig_macro['layout']['margin'].update({'b': 150})  
fig_macro['layout']['margin'].update({'t': 50})   



lista_bin_names = ["Bottom","Typical","Good","High","Top"]
lista_sections  =  ['Introduction', 'Methods', 'Results', 'Discussion']



# i use the axis title of the small plots as values of the axis of the macro plot
fig_macro['layout']['xaxis1'].update(title=lista_sections[0])  
fig_macro['layout']['xaxis2'].update(title=lista_sections[1])
fig_macro['layout']['xaxis3'].update(title=lista_sections[2])
fig_macro['layout']['xaxis4'].update(title=lista_sections[3])


fig_macro['layout']['yaxis1'].update(title=lista_bin_names[4])   # ojo con el orden aqui!!
fig_macro['layout']['yaxis2'].update(title=lista_bin_names[3])
fig_macro['layout']['yaxis3'].update(title=lista_bin_names[2])
fig_macro['layout']['yaxis4'].update(title=lista_bin_names[1])
fig_macro['layout']['yaxis5'].update(title=lista_bin_names[0])


    

offline.plot(fig_macro, auto_open=True, image = 'png', image_filename='multiplot_comparisons' ,image_width=3000, image_height=2200,
              filename='multiplot_comparisons.html', validate=True)




dict_group_subset_data.keys()

[[0, 2], [2, 8], [8, 25], [25, 79], [79, 1173]] (156558, 58)
[0, 2] (156558, 58) (35805, 58) [[(0.25, 7.0), (0.5, 12.0), (0.75, 17.0)], [(0.25, 1.0), (0.5, 3.0), (0.75, 6.0)], [(0.25, 0.0), (0.5, 0.0), (0.75, 2.0)], [(0.25, 5.0), (0.5, 11.0), (0.75, 18.0)]]
[12, 3, 0, 11] 
 [[7, 12, 17], [1, 3, 6], [0, 0, 2], [5, 11, 18]] 

[2, 8] (156558, 58) (57980, 58) [[(0.25, 8.0), (0.5, 13.0), (0.75, 18.0)], [(0.25, 1.0), (0.5, 3.0), (0.75, 6.0)], [(0.25, 0.0), (0.5, 0.0), (0.75, 4.0)], [(0.25, 6.0), (0.5, 12.0), (0.75, 19.0)]]
[13, 3, 0, 12] 
 [[8, 13, 18], [1, 3, 6], [0, 0, 4], [6, 12, 19]] 

[8, 25] (156558, 58) (46718, 58) [[(0.25, 9.0), (0.5, 13.0), (0.75, 19.0)], [(0.25, 1.0), (0.5, 3.0), (0.75, 6.0)], [(0.25, 0.0), (0.5, 1.0), (0.75, 6.0)], [(0.25, 7.0), (0.5, 13.0), (0.75, 20.0)]]
[13, 3, 1, 13] 
 [[9, 13, 19], [1, 3, 6], [0, 1, 6], [7, 13, 20]] 

[25, 79] (156558, 58) (14457, 58) [[(0.25, 9.0), (0.5, 13.0), (0.75, 19.0)], [(0.25, 1.0), (0.5, 3.0), (0.75, 7.0)], [(0.25, 0.0), (0.5, 2.0), 

dict_keys(['Introduction Bottom', 'Methods Bottom', 'Results Bottom', 'Discussion Bottom', 'Introduction Typical', 'Methods Typical', 'Results Typical', 'Discussion Typical', 'Introduction Good', 'Methods Good', 'Results Good', 'Discussion Good', 'Introduction High', 'Methods High', 'Results High', 'Discussion High', 'Introduction Top', 'Methods Top', 'Results Top', 'Discussion Top'])